In [35]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.axis as ax

In [36]:
data = pd.read_csv('churn_prediction.csv')
data.head()

,Unnamed: 0,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,-0.326221,0,0,0.293517,2,-1.225848,1,1,1,101348.88,1
1,1,-0.440036,0,0,0.198164,1,0.117350,1,0,1,112542.58,0
2,2,-1.536794,0,0,0.293517,8,1.333053,3,1,0,113931.57,1
3,3,0.501521,0,0,0.007457,1,-1.225848,2,0,0,93826.63,0
4,4,2.063884,0,0,0.388871,2,0.785728,1,1,1,79084.10,0


In [37]:
data['Gender']

0       0
1       0
2       0
3       0
4       0
       ..
9995    1
9996    1
9997    0
9998    1
9999    0
Name: Gender, Length: 10000, dtype: int64

In [38]:
del data['Unnamed: 0']

In [39]:
x =  data.iloc[:,0:10] # INPUT
y = data.iloc[:,-1] #output

In [40]:
x

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,-0.326221,0,0,0.293517,2,-1.225848,1,1,1,101348.88
1,-0.440036,0,0,0.198164,1,0.117350,1,0,1,112542.58
2,-1.536794,0,0,0.293517,8,1.333053,3,1,0,113931.57
3,0.501521,0,0,0.007457,1,-1.225848,2,0,0,93826.63
4,2.063884,0,0,0.388871,2,0.785728,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,1.246488,0,1,0.007457,5,-1.225848,2,1,0,96270.64
9996,-1.391939,0,1,-0.373958,10,-0.306379,1,1,1,101699.77
9997,0.604988,0,0,-0.278604,7,-1.225848,1,0,1,42085.58
9998,1.256835,1,1,0.293517,3,-0.022608,2,1,0,92888.52


### splitting the data

In [41]:
from sklearn.model_selection import train_test_split
x_train , x_test , y_train , y_test = train_test_split(x,y,test_size=0.3)

### Random Forest

In [42]:
from sklearn.ensemble import RandomForestRegressor
rfr = RandomForestRegressor(500)
rfr=rfr.fit(x_train , y_train)
predictions = rfr.predict(x_test)

In [43]:
predictions

array([0.   , 0.14 , 0.802, ..., 0.198, 0.14 , 0.01 ])

In [44]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test ,predictions)

0.11080188666666667

## <font color = 'red'> Stratified K-Fold cross Validation <font/>

In [45]:
from statistics import mean, stdev
from sklearn import preprocessing
from sklearn.model_selection import StratifiedKFold
from sklearn import linear_model

In [46]:
# from sklearn.model_selection import KFold
# kfold = KFold(n_splits=10 , shuffle=False)

In [47]:
# for train_index, test_index in kfold.split(x):
#     print("Train - \n" , train_index ,"\n Test - \n" , test_index)

#     print("===============================================\n")

In [48]:
# rf1 = RandomForestRegressor()

# n_est = [100 , 200 , 300 , 400 , 500 ]

# for n_estimators in n_est:
#     error=[]

#     for train_index, test_index in kfold.split(x):

#         x_train , x_test = x[train_index] , x[test_index]
#         y_train , y_test = y[train_index] , y[test_index]

#         rf1 = RandomForestRegressor(100)
#         rf1 = rf1.fit(x_train,y_train)

#         prediction = rf1.predict(x_test)
#         error.append(mean_squared_error(y_test , prediction))

#     print("Result for n_estimators : " , n_estimators)

#     print(error)

#     print("Average error = " , sum(error)/len(error))

#-------------------------------------------------------------------------------------------------------------------------------------------

# Feature Scaling for input features.
scaler = preprocessing.MinMaxScaler()
x_scaled = scaler.fit_transform(x)

# Create  classifier object.
lr = linear_model.LogisticRegression()

# Create StratifiedKFold object.
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1000)
lst_accu_stratified = []

for train_index, test_index in skf.split(x, y):
    x_train_fold, x_test_fold = x_scaled[train_index], x_scaled[test_index]
    y_train_fold, y_test_fold = y[train_index], y[test_index]
    lr.fit(x_train_fold, y_train_fold)
    lst_accu_stratified.append(lr.score(x_test_fold, y_test_fold))

# Print the output.
print('List of possible accuracy:', lst_accu_stratified)
print('\nMaximum Accuracy That can be obtained from this model is:',
      max(lst_accu_stratified)*100, '%')
print('\nMinimum Accuracy:',
      min(lst_accu_stratified)*100, '%')
print('\nOverall Accuracy:',
      mean(lst_accu_stratified)*100, '%')
print('\nStandard Deviation is:', stdev(lst_accu_stratified))

List of possible accuracy: [0.812, 0.809, 0.802, 0.81, 0.816, 0.823, 0.812, 0.83, 0.806, 0.795]

Maximum Accuracy That can be obtained from this model is: 83.0 %

Minimum Accuracy: 79.5 %

Overall Accuracy: 81.15 %

Standard Deviation is: 0.009980536614386545


## Applying the SMOT For imbalance dataset

In [49]:
print("Before OverSampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before OverSampling, counts of label '0': {} \n".format(sum(y_train == 0)))

# import SMOTE module from imblearn library
# pip install imblearn (if you don't have imblearn in your system)
from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state = 2)
X_train_res, y_train_res = sm.fit_resample(x_train, y_train.ravel())

print('After OverSampling, the shape of train_X: {}'.format(X_train_res.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_train_res.shape))

print("After OverSampling, counts of label '1': {}".format(sum(y_train_res == 1)))
print("After OverSampling, counts of label '0': {}".format(sum(y_train_res == 0)))



Before OverSampling, counts of label '1': 1399
Before OverSampling, counts of label '0': 5601 

After OverSampling, the shape of train_X: (11202, 10)
After OverSampling, the shape of train_y: (11202,) 

After OverSampling, counts of label '1': 5601
After OverSampling, counts of label '0': 5601


## Logistic Regression

In [51]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report


lr1 = LogisticRegression()
lr1.fit(X_train_res, y_train_res.ravel())
predictions = lr1.predict(x_test)

# print classification report
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       0.00      0.00      0.00      2362
           1       0.21      1.00      0.35       638

    accuracy                           0.21      3000
   macro avg       0.11      0.50      0.18      3000
weighted avg       0.05      0.21      0.07      3000



c:\Users\Ankur\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Ankur\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Ankur\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

## NearMiss Algorithm

In [53]:
print("Before Undersampling, counts of label '1': {}".format(sum(y_train == 1)))
print("Before Undersampling, counts of label '0': {} \n".format(sum(y_train == 0)))

# apply near miss
from imblearn.under_sampling import NearMiss
nr = NearMiss()

X_train_miss, y_train_miss = nr.fit_resample(x_train, y_train.ravel())

print('After Undersampling, the shape of train_X: {}'.format(X_train_miss.shape))
print('After Undersampling, the shape of train_y: {} \n'.format(y_train_miss.shape))

print("After Undersampling, counts of label '1': {}".format(sum(y_train_miss == 1)))
print("After Undersampling, counts of label '0': {}".format(sum(y_train_miss == 0)))


Before Undersampling, counts of label '1': 1399
Before Undersampling, counts of label '0': 5601 

After Undersampling, the shape of train_X: (2798, 10)
After Undersampling, the shape of train_y: (2798,) 

After Undersampling, counts of label '1': 1399
After Undersampling, counts of label '0': 1399


In [55]:
# train the model on train set
lr2 = LogisticRegression()
lr2.fit(X_train_miss, y_train_miss.ravel())
predictions = lr2.predict(x_test)

# print classification report
print(classification_report(y_test, predictions))


              precision    recall  f1-score   support

           0       0.89      0.72      0.80      2362
           1       0.39      0.67      0.50       638

    accuracy                           0.71      3000
   macro avg       0.64      0.70      0.65      3000
weighted avg       0.78      0.71      0.73      3000



## Applying the random forest for prediction 

In [56]:
# Fitting Random Forest Regression to the dataset
# import the regressor
from sklearn.ensemble import RandomForestRegressor

# create regressor object
regressor = RandomForestRegressor(n_estimators = 100, random_state = 0)

# fit the regressor with x and y data
regressor.fit(x, y)


RandomForestRegressor(random_state=0)

In [ ]:
Y_pred = regressor.predict(np.array([6.5]).reshape(1, 1)) # test the output by changing values


In [61]:

# Visualising the Random Forest Regression results

# arrange for creating a range of values
# from min value of x to max
# value of x with a difference of 0.01
# between two consecutive values
X_grid = np.arrange(min(x), max(x), 0.01)

# reshape for reshaping the data into a len(X_grid)*1 array,
# i.e. to make a column out of the X_grid value				
X_grid = X_grid.reshape((len(X_grid), 1))

# Scatter plot for original data
plt.scatter(x, y, color = 'blue')

# plot predicted data
plt.plot(X_grid, regressor.predict(X_grid),
		color = 'green')
plt.title('Random Forest Regression')
plt.xlabel('Position level')
plt.ylabel('Salary')
plt.show()


AttributeError: module 'numpy' has no attribute 'arrange'

In [62]:
from sklearn.trees import RandomForestClassifier
RandomeForest = RandomForestClassifier(oob_score=True)
RandomForest.fit(X_train,y_train)
print(RandomForest.oob_score_)


ModuleNotFoundError: No module named 'sklearn.trees'